Second stage - playing with API

In [ ]:
import sys
sys.path.append("../") 
from nebula_api.nebula_enrichment_api import NRE_API
nre = NRE_API()
db = nre.db
print(nre.db_host)

http://ec2-18-158-123-0.eu-central-1.compute.amazonaws.com:8529/


1) Scene detection using API. Copy new video to ~/upload

In [9]:
from nebula_api.scene_detector_api import NEBULA_SCENE_DETECTOR

scene_detector = NEBULA_SCENE_DETECTOR() 
movies= scene_detector.new_movies_batch_processing("/home/dimas/upload", "/home/dimas/tests", "development")
print(movies)

New movie found
1017_Bad_Santa_00.53.52.592-00.53.58.245.avi
1017_Bad_Santa_00


 17%|█▋        | 19/114 [00:00<00:00, 185.75frames/s]

DEBUG:  /home/dimas/tests/1017_Bad_Santa_00.mp4


 18%|█▊        | 20/114 [00:00<00:00, 195.54frames/s]

Scenes:  [[0, 114]]


100%|██████████| 114/114 [00:00<00:00, 229.43frames/s]


Scenes:  [[0, 114]]
TODO
{'movie_id': '29744e1ba67e438aa078b2fd022dea45', 'file_name': '1017_Bad_Santa_00.53.52.592-00.53.58.245.avi', 'movie_name': '1017_Bad_Santa_00', 'description': '1017_Bad_Santa_00', 'tags': ['lsmdc', 'pegasus', 'visual genome'], 'full_path': '/home/dimas/tests/1017_Bad_Santa_00.mp4', 'url': 'static/development/1017_Bad_Santa_00.mp4', 'last_frame': 300, 'scenes': [[0, 114]], 'scene_elements': [[0, 114]], 'mdfs': [[3, 60, 110]], 'metadata': {'width': 1920, 'height': 1080, 'fps': 23}, 'status': 'created'}
[('/home/dimas/upload/1017_Bad_Santa_00.53.52.592-00.53.58.245.avi', 'Movies/121081267')]


Upload frames to S3 for future processing (experts)

In [ ]:
for movie in movies:
    scene_detector.store_frames_to_s3(movie[1], "/dataset/frames/", movie[0])

/dataset/frames/


2) Create StoryLine - get CLIP sentences and triplets using MilvusAPI from Milvus vector DB and create related entities in StoryLine collection of ArangoDB (see milvus_api for)

In [ ]:
from nebula_api.story_line_api import STORY_LINE_API
story_line = STORY_LINE_API() 
meta_ = story_line.get_movie_meta(movie[1])
print(meta_)
meta = meta_[movie[1]]
print("Processing Movie: ", movie)
for i, scene_element in enumerate(meta['scene_elements']):
    file_name = meta['full_path']
    movie_id = meta['movie_id']
    arango_id = meta['_id']
    mdfs = meta['mdfs'][i]
    start_frame = scene_element[0]
    stop_frame = scene_element[1]
    stage = i
    story_line.create_story_line(file_name, movie_id, arango_id, stage, start_frame, stop_frame, mdfs)
            

Milvus Collection Loaded:  scene_graph_visual_genome
Milvus Collection Loaded:  pegasus
{'Movies/121081267': {'_key': '121081267', '_id': 'Movies/121081267', '_rev': '_dJbDATW---', 'movie_id': '699b48742ce04a76af4d8acb87df8d64', 'file_name': '1017_Bad_Santa_00.53.52.592-00.53.58.245.avi', 'movie_name': '1017_Bad_Santa_00', 'description': '1017_Bad_Santa_00', 'tags': ['lsmdc', 'pegasus', 'visual genome'], 'full_path': '/home/dimas/tests/1017_Bad_Santa_00.mp4', 'url_path': 'static/development/1017_Bad_Santa_00.mp4', 'last_frame': 300, 'scenes': [[0, 136]], 'scene_elements': [[0, 115], [115, 136]], 'mdfs': [[3, 61, 112], [118, 128, 132]], 'meta': {'fps': 23, 'height': 1080, 'width': 1920}, 'updates': 3, 'status': 'created'}}
Processing Movie:  ('/home/dimas/upload/1017_Bad_Santa_00.53.52.592-00.53.58.245.avi', 'Movies/121081267')
/home/dimas/tests/1017_Bad_Santa_00.mp4
Scene:  0
/home/dimas/tests/1017_Bad_Santa_00.mp4
Scene:  1


Add metadata to ElasticSearch, make it possible to search in WebUI by keywords

In [11]:
from nebula_api.elastic_index_api import ELASTIC_SEARCH
es = ELASTIC_SEARCH()
print("Processing Movie: ", movie[1])
es.insert_update_index(movie[1])
print("Index created for: ", movie[1])

2021-10-25 12:04:33,272 - POST http://ec2-18-158-123-0.eu-central-1.compute.amazonaws.com:9200/datadriven/_doc [status:201 request:0.005s]
2021-10-25 12:04:33,275 - POST http://ec2-18-158-123-0.eu-central-1.compute.amazonaws.com:9200/datadriven/_search [status:200 request:0.002s]


Processing Movie:  Movies/121081267
ID:  BHFVt3wB8pdCmyP2yBJx
Found existing index for  Movies/121081267  updating
DEBUG:  FOR doc IN Movies FILTER doc._id == "Movies/121081267"  RETURN doc
DEBUG:  {'movie_name': '1017_Bad_Santa_00', 'video': '/static/development/1017_Bad_Santa_00.mp4', 'url': '/static/development/1017_Bad_Santa_00.mp4', 'slice_interval': '00:00-00:04', 'timestamp': '0', 'description': 'man red truck . character deliver doctor standing ambulance body electric golf cart get entity guide jump start car classic scooter jacket clean actor mystery holiday flag mayor american football safety tackle vacation artist motorcycle undisputed king movie snowsuit wear white santa claus wheel mailman bend mail cross guard vest green blazer crossing blade tip residue', 'tags': ['lsmdc', 'pegasus', 'visual genome'], 'main_tags': ['man', 'red', 'snowsuit', 'man', 'wear', 'red', 'white', 'jacket', 'man', 'red', 'jacket', 'santa', 'claus', 'motorcycle', 'santa', 'wear', 'jacket', 'man', '

2021-10-25 12:04:33,481 - POST http://ec2-18-158-123-0.eu-central-1.compute.amazonaws.com:9200/datadriven/_update/BHFVt3wB8pdCmyP2yBJx [status:200 request:0.008s]


Index created for:  Movies/121081267


Please see SceneGraph.py and StoryGraph.py for different GRAPH creation strategies:
Examples:
1) scene_graph.create_scene_graph(movie) # Create triplet-based scene for one scene element graph
2) story_graph.create_story_graph(movie) # Connect all scene element in one movie graph